In [2]:
import sys
sys.path.append('/opt/ml/level2_nlp_datacentric-nlp-09')
from preprocessing.data_cleaning import g2p_cleaning

from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
from transformers import AutoModelForSequenceClassification
from tokenization_kobert import KoBertTokenizer
from cleanlab.filter import find_label_issues
from transformers import AutoModelForSequenceClassification, AutoConfig

In [3]:
train = pd.read_csv('../data/train_v0.12.1.csv')
valid = pd.read_csv('../data/valid_v0.12.1.csv')
dataset_total = pd.concat([train, valid], axis=0)

# 모델 체크포인트 파일과 구성 설정 파일 경로
config_file = "../output/checkpoint-2000/config.json"
model_checkpoint = "../output/checkpoint-2000/pytorch_model.bin"

# 구성 설정 파일 로드
config = AutoConfig.from_pretrained(config_file)

# 토크나이저 및 모델 로드
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, config=config)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [4]:
probs = []
print("\n[Cleanlab Label Error Detection]")
for _, dt in tqdm(dataset_total.iterrows(), total=len(dataset_total)):
    # 입력 문장
    input_text = dt["text"]

    # 입력 문장 토큰화 및 패딩
    input_ids = tokenizer.encode(input_text, add_special_tokens=True)
    input_ids = torch.tensor([input_ids])
    # print(input_ids)
    # 모델에 입력 전달
    outputs = model(input_ids)

    # 로짓(마지막 레이어 출력값)과 클래스 확률 계산
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)

    # 클래스 확률 출력
    class_probabilities = probabilities[0].tolist()
    # print(class_probabilities)
    probs.append(class_probabilities)


[Cleanlab Label Error Detection]


100%|██████████| 45678/45678 [1:09:06<00:00, 11.02it/s]


In [5]:
label_dict = {0:"IT과학", 1:"경제", 2:"사회", 3:"생활문화", 4:"세계", 5:"스포츠", 6:"정치"}

# cleanlab
train_pred_probs = torch.Tensor()

ordered_label_issues = find_label_issues(
    labels=dataset_total["target"],
    pred_probs=np.array(probs),
    return_indices_ranked_by="self_confidence",
)

print("\n[Print Changed Sentences and Labels]")
count = 0
for issue in ordered_label_issues:
    count += 1
    print(issue, dataset_total.iloc[issue]["text"],"|",
          label_dict[dataset_total.iloc[issue]["target"]],
          dataset_total.iloc[issue]["target"])
    # print(probs[issue])
print(count)


[Print Changed Sentences and Labels]
20442 김종대 美 北단거리 미사일 묵인…새끼미사일 제재한 적 없어 | 생활문화 3
38768 北주도 태권도연맹 프란치스코 교황에 명예9단 수여 | 경제 1
35691 北노동신문 협상진전 바라는 美 제재문제 알아서 처리해야 | 생활문화 3
24708 朴대통령 시릴만큼 아프게 부모잃어…안보소명론으로 사드호소종합 | 생활문화 3
40863 이해찬 철도 공동조사 내주 시작…남북관계 큰 변화 | 생활문화 3
43001 평양정상회담 D1 文대통령 관람할 환영공연 뭘까…이목 집중 | 경제 1
41380 朴대통령 北風 음모론 가슴아파…내부로 칼끝 돌려선 안돼 | 경제 1
20707 안철수 北미사일 대응기술 예산 필요하면 적극 협조 | 생활문화 3
34908 문 대통령김정은 MDL서 역사적 만남…반갑게 악수속보 | 경제 1
35953 대통령 개헌안 대통령의 헌재소장 임명권 조항 삭제속보 | 경제 1
21219 북한인권정보센터 北 인권범죄 절반 자의적 체포와 구금 | IT과학 0
23932 2野 靑회동서 사드 문제 발언 수위 고심 | 경제 1
24485 수상 스포츠의류 제조업체 배럴 내달 1일 코스닥 상장 | 정치 6
36149 평양공동선언 정의 전쟁 종식의 문 열려…미국도 적극 응답하라 | IT과학 0
39913 이총리 방일 위해 출국…2박3일 한일 디딤돌 외교 나선다 | IT과학 0
32638 황총리 불법 선거운동·사이버 선거범죄 강력 대처 | 경제 1
10255 메리츠증권 게임빌 풍성한 신작…실적개선 기대 | 정치 6
3012 北 당대회 앞두고 끝내지 않은 공사도 준공식 RFA | 생활문화 3
34371 北김정은 軍 중대성명 발표 이틀만에 150여만명 입대 지원 | 경제 1
38801 박지원 秋 사드반대 당론땐 천군만마…野공조로 특위 구성 | 경제 1
27143 지옥에서 돌아온 고동진 갤럭시S8 대박에 활짝 | 정치 6
14449 SK텔레콤 본사서 5G스마트폰 전시 | 정치 6
40819 그래픽 201

In [11]:
ordered_label_issues

array([20442, 38768, 35691, ..., 32160, 36396, 36108])

In [19]:
# 예측값으로 학습데이터 재 라벨링
pd.set_option('mode.chained_assignment', None)

total_converted = dataset_total.copy()
for oli in ordered_label_issues[:1500]:
    prob = probs[oli]
    total_converted.target.iloc[oli] = prob.index(max(prob))

    print(oli, total_converted.iloc[oli]["text"], "|",
          label_dict[dataset_total.iloc[oli]["target"]],
          dataset_total.iloc[oli]["target"], "->",
          label_dict[total_converted.iloc[oli]["target"]],
          total_converted.iloc[oli]["target"])

20442 김종대 美 北단거리 미사일 묵인…새끼미사일 제재한 적 없어 | 생활문화 3 -> 정치 6
38768 北주도 태권도연맹 프란치스코 교황에 명예9단 수여 | 경제 1 -> 정치 6
35691 北노동신문 협상진전 바라는 美 제재문제 알아서 처리해야 | 생활문화 3 -> 정치 6
24708 朴대통령 시릴만큼 아프게 부모잃어…안보소명론으로 사드호소종합 | 생활문화 3 -> 정치 6
40863 이해찬 철도 공동조사 내주 시작…남북관계 큰 변화 | 생활문화 3 -> 정치 6
43001 평양정상회담 D1 文대통령 관람할 환영공연 뭘까…이목 집중 | 경제 1 -> 정치 6
41380 朴대통령 北風 음모론 가슴아파…내부로 칼끝 돌려선 안돼 | 경제 1 -> 정치 6
20707 안철수 北미사일 대응기술 예산 필요하면 적극 협조 | 생활문화 3 -> 정치 6
34908 문 대통령김정은 MDL서 역사적 만남…반갑게 악수속보 | 경제 1 -> 정치 6
35953 대통령 개헌안 대통령의 헌재소장 임명권 조항 삭제속보 | 경제 1 -> 정치 6
21219 북한인권정보센터 北 인권범죄 절반 자의적 체포와 구금 | IT과학 0 -> 정치 6
23932 2野 靑회동서 사드 문제 발언 수위 고심 | 경제 1 -> 정치 6
24485 수상 스포츠의류 제조업체 배럴 내달 1일 코스닥 상장 | 정치 6 -> 경제 1
36149 평양공동선언 정의 전쟁 종식의 문 열려…미국도 적극 응답하라 | IT과학 0 -> 정치 6
39913 이총리 방일 위해 출국…2박3일 한일 디딤돌 외교 나선다 | IT과학 0 -> 정치 6
32638 황총리 불법 선거운동·사이버 선거범죄 강력 대처 | 경제 1 -> 정치 6
10255 메리츠증권 게임빌 풍성한 신작…실적개선 기대 | 정치 6 -> 경제 1
3012 北 당대회 앞두고 끝내지 않은 공사도 준공식 RFA | 생활문화 3 -> 정치 6
34371 北김정은 軍 중대성명 발표 이틀만에 150여만명 입대 지원 | 경제 1 -> 정치 6
38801 박지원 秋 사드반

In [15]:
total_converted

,ID,text,label_text,target,url,date
0,ynat-v1_train_34661,홍콩 행정장관 유화 제스처…시위대 요구 일부 수용 시사종합,세계,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.21. 오후 10:49
1,ynat-v1_train_26526,신간 한 문장·그대에게 연을 띄우며,생활문화,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.17. 오전 7:00
2,ynat-v1_train_20200,文대통령여야 대표 4개월만에 회동…꽉 막힌 정국 풀릴까,스포츠,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.11.08. 오후 12:45
3,ynat-v1_train_13212,퇴임 앞둔 英총리 트럼프 만나 기후변화 문제제기할 것,세계,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.03. 오후 4:41
4,ynat-v1_train_26282,웃음바다 된 MLB 감독 기자회견 트럼프 대통령 탄핵 찬...,IT과학,5,https://sports.news.naver.com/news.nhn?oid=001...,2019.10.07 07:58
...,...,...,...,...,...,...
13699,ynat-v1_train_45498,움짜 류행에 갤노트8도 가세…S펜으로 직접 만들고 공유종합,정치,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.08.24. 오전 3:55
13700,ynat-v1_train_45543,수원 화성문화제·정선아리랑제 등 새해 문화관광축제 35개 선정,생활문화,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.27. 오전 10:47
13701,ynat-v1_train_45594,AI 스피커 카카오미니 이달 중 예약판매 개시,정치,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.09.05. 오전 11:11
13702,ynat-v1_train_45603,새로운 시작 화합과 나누…2018 신년음악회,생활문화,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.01.08. 오후 12:12


In [16]:
train_g2p_cleaned = pd.read_csv('../data/train_v0.12.1.csv')

print("\n[Train Eval Splitting]")
train_g2p_label_cleaned = pd.DataFrame()
valid_g2p_label_cleaned = pd.DataFrame()
for idx, tc in tqdm(total_converted.iterrows(), total=len(total_converted)):
    if tc.ID in train_g2p_cleaned.ID.tolist():
        train_g2p_label_cleaned = pd.concat([train_g2p_label_cleaned, pd.DataFrame(tc).transpose()], ignore_index=True)
    else:
        valid_g2p_label_cleaned = pd.concat([valid_g2p_label_cleaned, pd.DataFrame(tc).transpose()], ignore_index=True)


[Train Eval Splitting]


100%|██████████| 45678/45678 [05:39<00:00, 134.70it/s]


In [17]:
print("Train Length:", len(train_g2p_label_cleaned))
print("Valid Length:", len(valid_g2p_label_cleaned))
train_g2p_label_cleaned.to_csv('../data/train_v0.12.4.csv', index=False)
valid_g2p_label_cleaned.to_csv('../data/valid_v0.12.4.csv', index=False)
print("# of Changed Target in Train:", sum(train.target != train_g2p_label_cleaned.target))
print("# of Changed Target in Valid:", sum(valid.target != valid_g2p_label_cleaned.target))

Train Length: 31974
Valid Length: 13704
# of Changed Target in Train: 1011
# of Changed Target in Valid: 489
